# Fome Zero Foods - Visão País

Neste notebook vamos explorar o dataset de restaurantes da Fome Zero Foods, respondendo a perguntas de negócios sobre o contexto da visão país do dataset.

## 1. Importando bibliotecas e criando Dataframe

### 1.1 Importando bibliotecas

In [16]:
import pandas as pd
import plotly.express as px

from pathlib import Path

### 1.2 Criando Dataframe

In [17]:
# caminho do arquivo
path = Path.cwd().parent / 'data' / 'processed' / 'dataset_processed.csv'

try:
    with open(path, 'r', encoding='utf-8') as arquivo:
        df = pd.read_csv(arquivo)
except FileNotFoundError:
    print(f"Arquivo não encontrado: {path}")

In [18]:
df.head(3)

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_online_delivery,is_delivering_now,price_range,aggregate_rating,rating_color,rating_text,votes,country_name,price_type,color_name
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,Philippines,expensive,darkgreen
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,Philippines,gourmet,darkgreen


## 2. Perguntas de Negócio

### 2.1 Qual o nome do país que possui mais cidades registradas?

In [46]:
df_name = (
    df.loc[:, ['city', 'country_name']]
    .groupby('country_name')
    .nunique()
    .sort_values(by='city', ascending=False)
    .reset_index()
).head(10)

#print(f"País com maior quantidade de cidades: {df_name.iloc[0,0]}, com {df_name.iloc[0,1]} cidades.")
fig = px.bar(df_name, x='country_name', y='city')
fig.show()


### 2.2 Qual o nome do país que possui mais restaurantes registrados?

In [47]:
df_rest = (
    df.loc[:, ['restaurant_name', 'country_name']]
    .groupby('country_name')
    .nunique()
    .sort_values(by='restaurant_name', ascending=False)
    .reset_index()
).head(10)

#print(f"País com maior quantidade de cidades: {df_rest.iloc[0,0]}, com {df_rest.iloc[0,1]:,} restaurantes.")
px.bar(df_rest, x='country_name', y='restaurant_name')

### 2.3 Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?

In [21]:
filtro = df['price_range'] == 4

df_country = (
    df.loc[filtro, ['restaurant_id', 'country_name']]
    .groupby('country_name')
    .nunique()
    .sort_values(by='restaurant_id', ascending=False)
    .reset_index()
)

print(f" País com maior quantidade de restaurantes com nível de preço 4: {df_country.iloc[0,0]}")

 País com maior quantidade de restaurantes com nível de preço 4: United States of America


### 2.4 Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

In [22]:
df_cuisines = (
    df.loc[:, ['cuisines', 'country_name']]
    .groupby('country_name')
    .nunique()
    .sort_values(by='cuisines', ascending=False)
    .reset_index()
)

print(f"País com a maior quantidade de culinarias: {df_cuisines.iloc[0,0]}, com {df_cuisines.iloc[0,1]} culinarias. ")

País com a maior quantidade de culinarias: India, com 77 culinarias. 


### 2.5 Qual o nome do país que possui a maior quantidade de avaliações feitas?

In [23]:
pais = (
    df.loc[:, ['country_name', 'aggregate_rating', 'restaurant_id']]
    .groupby('country_name')
    .agg(count_rating=('aggregate_rating', 'count'), rest_id_min=('restaurant_id', 'min'))
    .sort_values(by=['count_rating', 'rest_id_min'], ascending=[False, True])
    .reset_index()
)
print(f"País que possuí a maior quantidade de avaliações: {pais.iloc[0,0]}")

País que possuí a maior quantidade de avaliações: India


### 2.6 Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

In [24]:
pais = (
    df.loc[df['is_delivering_now'] == 1, ['country_name', 'is_delivering_now']]
    .groupby('country_name')
    .count()
    .sort_values('is_delivering_now', ascending=False)
    .reset_index()
    )

print(f"País com mais restaurantes realizando entregas: {pais.iloc[0,0]}")

País com mais restaurantes realizando entregas: India


### 2.7 Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

In [25]:
pais = (
    df.loc[df['has_table_booking'] == 1, ['country_name', 'has_table_booking']]
    .groupby('country_name')
    .count()
    .sort_values('has_table_booking', ascending=False)
    .reset_index()
    )

print(f"País com mais restaurantes que aceitam reservas: {pais.iloc[0,0]}")

País com mais restaurantes que aceitam reservas: India


### 2.8 Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

In [26]:
pais = (
    df.loc[:, ['country_name', 'aggregate_rating']]
    .groupby('country_name')
    .count()
    .sort_values('aggregate_rating', ascending=False)
    .reset_index()
)

print(f"País com maior quantidade de avaliações registradas: {pais.iloc[0,0]}")

País com maior quantidade de avaliações registradas: India


### 2.9 Qual o nome do país que possui, na média, a maior nota média registrada?

In [27]:
pais = (
    df.groupby('country_name')['aggregate_rating']
    .mean()
    .sort_values(ascending=False)
    .reset_index()
)

print(f"País com maior nota média nas avaliações registradas: {pais.iloc[0,0]}")

País com maior nota média nas avaliações registradas: Indonesia


### 2.10 Qual o nome do país que possui, na média, a menor nota média registrada?

In [28]:
pais = (
    df.groupby('country_name')['aggregate_rating']
    .mean()
    .sort_values(ascending=True)
    .reset_index()
)

print(f"País com menor nota média nas avaliações registradas: {pais.iloc[0,0]}")

País com menor nota média nas avaliações registradas: Brazil


### 2.11 Qual a média de preço de um prato para dois por país?

In [29]:
preco_medio = (
    df.loc[:, ['country_name', 'average_cost_for_two']]
    .groupby('country_name')
    .mean()
    .sort_values('average_cost_for_two', ascending=False)
    .reset_index()
)

preco_medio

,country_name,average_cost_for_two
0,Indonesia,312682.926829
1,Sri Lanka,2607.471264
2,Philippines,1211.090909
3,India,703.012014
4,South Africa,335.479058
5,Qatar,176.648936
6,United Arab Emirates,158.218563
7,Singapure,141.036585
8,Brazil,137.375479
9,Turkey,128.848485
